In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
# from tensorflow.keras.preprocessing.text import Tokenizer,one_hot
from tensorflow.keras.preprocessing import sequence
# import string
from tensorflow.keras.layers import  Embedding,Dense,Conv1D,Input,Flatten,LeakyReLU,Concatenate
from tensorflow.keras.models import Model
import os


# اخبار

In [2]:
news = pd.read_csv("E:\\every_thing_about_python\\rahnamaKalej\\machine learning bootcamp\\DataSets\\MINDlarge_train\\news.tsv"
        ,sep='\t',header=None,)

news.columns  =["NewsID"
        ,"Category","SubCategory","Title","Abstract"
            ,"URL","Title Entities","Abstract Entities"]

# کاربران

In [3]:
users = pd.read_csv("E:\\every_thing_about_python\\rahnamaKalej\\machine learning bootcamp\\DataSets\\MINDlarge_train\\behaviors.tsv"
        ,sep='\t',header=None,)

users.columns = ["impression ID","UserID","Time","History","Impressions"]

# (فیلتر اول) ساخت کلاس برای لایه‌ی اول معماری

In [5]:
class Favorite_Categories():

    def __init__(self , users ,news):

        
        self.news                   = (news.copy(deep=True)).set_index("NewsID")



        self.users                  = users.copy(deep=True)
        self.number_of_categoeies   = len(set(self.news.Category))
        
        self.users.fillna(" ",inplace=True)
        
        

        self.category_of_news       = list(set(self.news.Category))


    def get_favorite_categories_in_term(self,userid):

        #  تاریخچه ی اخبار مشاهده شده توسط کاربر
        
        history                     = " ".join(set(self.users[self.users.UserID==f"U{userid}"].History)).split()
    

        category_history            = [ self.news.loc[news].Category    for news in history]
        category_history            = ' '.join(category_history)
        
        favorite_dict               ={ f"{category}":category_history.count(category) 
                                        for category in self.category_of_news if category in category_history }


        if favorite_dict:
            
            #  در این لیست ، دسته ی اخباری که مشاهده شده اند، به ترتیب بیشترین بازدید تا کمترین بازدید چیده می شوند
            favorit_list            =[]

            while favorite_dict:

                max_seen_category   = max( favorite_dict ,key=favorite_dict.get )
                favorite_dict.pop(max_seen_category)

                favorit_list.append(max_seen_category)

            return favorit_list
        
        else:


            return  self.category_of_news
            



        
Filter1 = Favorite_Categories(users ,news)

    


# RL_RS تعریف محیط برای لایه‌ی دوم معماری 

In [6]:
class SecondFilterEnvironment():
                                    
    def __init__(self,users , news ,max_news_features=5000):
        
        # max_feature_news ==> در یک متن پیدا میکند Tfidf حداکثر تعداد لغاتی که بردار 

        # تمامی کاربران
        self.users = users.copy(deep=True)
        # خالی قرار داده میشود  String  یک  NAN برای کاربرانی که تاریخچه ای ندارند، به جای 
        self.users.History.fillna(" ",inplace=True)
        
        self.news = (news.copy(deep=True)).set_index("NewsID")
        self.number_of_categoeies = len(set(self.news.Category))

        #  تمامی کلماتی که برای پردازش نیاز داریم 
        self.AllWords =" ".join(set(self.news.Category)) + " ".join(set(self.news.SubCategory)) + " ".join(self.news.Title)

        self.NewsToArray = TfidfVectorizer(max_features=max_news_features,stop_words="english")
        self.NewsToArray.fit([self.AllWords])




        
       



    # DO NOT USE THIS METHOD!!!!!!!!!!!!!!!!!!!!!!!!!!!!   
    # this is a private method that gives state  in vector form and Text form
    def __get_state(self,userid ):

        UserHistory     = " ".join(set(self.users[self.users.UserID==f"U{userid}"].History)).split()
        
        Category        = ""
        
        SubCategory     = ""

        Title           = ""
        
        for news in UserHistory:

            Category   += " " + self.news.loc[news].Category
            SubCategory+= " " + self.news.loc[news].SubCategory
            Title      += " " + self.news.loc[news].Title

        # با کمک این بردار میتوان به ویژگی های متون خبری ای که کاربر قبلا مشاهده کرده دست یافت
        UserFeaturesText = Category +" "+ SubCategory +" "+ Title
        UserFeaturesArray = self.NewsToArray.transform([UserFeaturesText])
        UserFeaturesArray = UserFeaturesArray.toarray()*100


        return UserFeaturesText , UserFeaturesArray
    
    


    # DO NOT USE THIS METHOD!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # this is a private method 
    def __get_state_actionReward_newState(self,userid ):
        
        StateText ,StateVector                    = self.__get_state(userid)


        #  تمامی اخباری که به کاربر پیشنهاد شده است.
        Impressions                               = " ".join(self.users[self.users.UserID==f"U{userid}"].Impressions).split()

        #  تعریف یک دیکشنری برای این که مشخض شود هر کدام از اخبار پیشنهادی به کاربر، دیده شده است یا خیر
        ImpressionsDict                           = { im.split("-")[0]:int(im.split("-")[-1])     for im in Impressions}
        # /////////////////////
        
        # به روز رسانی گذشته ی کاربر
        NewStateText                              =  ""
        NewStateText                              += StateText

        LikedNewsText                             =  ""
        DisLikedNewsText                          =  ""

        Reward                                    =  19
        PunishMent                                =  -8

        #  به ازای هر خبری که مشاهده شده 19 امتیاز مثبت و به ازای هر خبری که مشاهده نشده 8 امتیاز منفی دریافت میشود
        RecievedRewards                           = {"Punishment":0 ,"Reward":0}

        for news,seen in ImpressionsDict.items():
            
            Category                              = self.news.loc[news].Category
            Subcategory                           = self.news.loc[news].SubCategory
            Title                                 = self.news.loc[news].Title

            NewsFeatures                          = Category + Subcategory + Title

            #به روز رسانی میشود newstate اگر کاربر خبر را مشاهده کرده باشد، پس بردار مربوط به ویژگی های کاربر به در غالب 
            # اضافه میشود  LikedNews  همچنین کلمات استفاده شده در این خبر برای کشف ویژگی های خبر مورد علاقه ی کاربر به متغیر
            if seen==1:
                NewStateText                      += " "      + NewsFeatures
                LikedNewsText                     += " "      + NewsFeatures 
                RecievedRewards["Reward"]         += Reward
            
            else:
                DisLikedNewsText                  += " "        + NewsFeatures
                RecievedRewards["Punishment"]     += PunishMent

        # گذشته ی به روز رسانی شده ی کاربر
        NewStateVector                             = self.NewsToArray.transform([NewStateText])
        NewStateVector                             = NewStateVector.toarray()*100
        # ویژگی های متون خبری ای که کاربر دوست داشته
        LikedNewsVector                            = self.NewsToArray.transform([LikedNewsText])
        LikedNewsVector                            = LikedNewsVector.toarray()*100
        # ویژگی های متون خبری ای که کاربر دوست نداشته
        DisLikedNewsVector                         = self.NewsToArray.transform([DisLikedNewsText])
        DisLikedNewsVector                         = DisLikedNewsVector.toarray()*100

        Result                                     ={ "state":StateVector , "LikedNewsVector":LikedNewsVector ,
                                                      "DisLikedNewsVector":DisLikedNewsVector ,"newstate":NewStateVector  }
        
        
        return {**Result , **RecievedRewards}
    

    #  ONLY USE THIS METHOD*****************************
    def get_transitions(self,userid):

        Transitions =  self.__get_state_actionReward_newState(userid)

        return Transitions



env = SecondFilterEnvironment(users , news ,5000)



del users , news

#   ساخت مدل 

In [12]:
def create_model(StateShapeUser,StateShapeNews):
    

    UserFeatures = Input(shape=StateShapeUser,name="User Features")

    NewsFeatures = Input(shape=StateShapeNews,name="News Features")

    user         = Conv1D(128,1,input_shape=(1,5000))(UserFeatures)
    user         = LeakyReLU()(user)
    user         = Flatten()(user)
    user         = Dense(128)(user)
    user         = LeakyReLU()(user)


    news         = Conv1D(128,1)(NewsFeatures)
    news         = LeakyReLU()(news)
    news         = Flatten()(news)
    news         = Dense(128)(news)
    news         = LeakyReLU()(news)


    concate      = Concatenate()([user ,news])

    concate      = Dense(256)(concate)
    concate      = LeakyReLU()(concate)

    out          = Dense(2)(concate)




    model        = Model(inputs=[UserFeatures,NewsFeatures],outputs=out) 

    model.summary()

    return model



UserStateShape  =(1,5000)
NewsStateShape  =(1,5000)

model           = create_model( UserStateShape , NewsStateShape )
target_model    = create_model( UserStateShape , NewsStateShape )

target_model.set_weights(model.get_weights())

#//////////////  loss_function( y_true , y_predict )
loss_function = tf.keras.losses.Huber()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)



Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 User Features (InputLayer)     [(None, 1, 5000)]    0           []                               
                                                                                                  
 News Features (InputLayer)     [(None, 1, 5000)]    0           []                               
                                                                                                  
 conv1d_16 (Conv1D)             (None, 1, 128)       640128      ['User Features[0][0]']          
                                                                                                  
 conv1d_17 (Conv1D)             (None, 1, 128)       640128      ['News Features[0][0]']          
                                                                                            

#  ساخت یک کلاس برای آموزش راحت تر مدل

In [25]:

class Dqn_model():

    def __init__(self,env,model ,target_model ,loss_function ,optimizer ,discount_factor ,action_size= 2):

        self.env            = env
        self.action_size    = action_size
        self.model          = model
        self.target_model   = target_model
        self.loss_function  = loss_function
        self.optimizer      = optimizer
        self.discount       = discount_factor
        
    # private method
    #   this method is the core to learn policy
    @tf.function
    def __train(self,StateUser,StateNews ,action ,reward ,NewStateUser):

        with tf.GradientTape() as tape:

            Q1 = self.model([StateUser ,StateNews] ,training=True)
            Q2 = Q1[::]

            reward = tf.cast(reward ,tf.float32)
            Y = reward + self.discount * tf.reduce_max( self.target_model([NewStateUser,StateNews] , training=True) ,axis=-1 )
            
            
            action_index = tf.one_hot([action] , self.action_size)

            Q2 = tf.tensor_scatter_nd_update(Q2 , tf.where(action_index)  , Y)
            
            loss = self.loss_function(Q2 ,Q1)

        grad = tape.gradient(loss , self.model.trainable_variables)
        self.optimizer.apply_gradients(zip( grad , self.model.trainable_variables))
        
        return loss

    # private method
    #  This method helps to put the above method in a for loop
    def __learn(self,StateUser,StateNews ,action ,reward ,NewStateUser ):
        
        loss = self.__train(StateUser,StateNews ,action ,reward ,NewStateUser)

        return loss
        
    # به روز رسانی شبکه هدف
    @tf.function
    def soft_Update(self,  target_model_weights    ,  model_weights   ,  tau=0.01):

        for TM , M in zip(target_model_weights ,model_weights):
            TM.assign(  (1-tau)*TM  +  tau * M   )
        

    #  این تابع ، با استفاده از  محیط، به شبکه ی عصبی، ویژگی های اخبار مورد علاقه ی و  کاربر مورد نظر را آموزش می دهد
    def Learn_favorite_category(self,userid , n_epocks=1 ,early_stopping = 0.01):

        transition          = self.env.get_transitions(userid) 

        StateUser           = transition["state"][np.newaxis,:,:]
        NewStateUser        = transition["newstate"][np.newaxis,:,:]
        LikedNewsVector     = transition["LikedNewsVector"][np.newaxis,:,:]
        DisLikedNewsVector  = transition["DisLikedNewsVector"][np.newaxis,:,:]
        ActionForLike       = 1
        ActionForDislike    = 0

        Reward              = transition["Reward"]
        Punishment          = transition["Punishment"]

        TotalLoss=[]
        for epock in range(n_epocks):
            
            loss1          = self.__learn(StateUser,LikedNewsVector    ,ActionForLike    ,Reward     ,NewStateUser )
            loss2          = self.__learn(StateUser,DisLikedNewsVector ,ActionForDislike ,Punishment ,NewStateUser ,)
            
            TotalLoss.append(loss1+loss2)
            
            if (loss1 + loss2)<=early_stopping:
                break        
                
        MeanTotalLoss     = np.mean(TotalLoss)

        return MeanTotalLoss

        





dqn                     = Dqn_model(env ,model ,target_model ,loss_function ,optimizer,0.9)




#  debugging for Env and DQN classes......

In [26]:
for i in np.random.randint(low =0 ,high = 711221,size= 50):
    
    #می‌باشد  __train   در بالای تابع   @tf.function    مشاهده شد، مربوط به    warning  اگر در این جا 
    print(f"dqn is learning User{i}")
    dqn.Learn_favorite_category(i)

dqn is learning User689161
dqn is learning User187914
dqn is learning User161422
dqn is learning User644294
dqn is learning User126109
dqn is learning User347246
dqn is learning User374195
dqn is learning User675577
dqn is learning User511749
dqn is learning User332176
dqn is learning User165282
dqn is learning User473475
dqn is learning User489641
dqn is learning User413659
dqn is learning User110273
dqn is learning User171397
dqn is learning User68950
dqn is learning User55793
dqn is learning User188503
dqn is learning User403105
dqn is learning User157178
dqn is learning User129343
dqn is learning User70726
dqn is learning User603192
dqn is learning User65371


# train DQN

In [27]:
n_episodes        = 10   #     تعداد کل مراحل آموزشی 
length_of_episode = 10   #     در هر بار آموزش شبکه عصبی، چند کاربر به شبکه عصبی آموزش داده میشود

n_iterations      = 10   #     علایق هر کاربر، چند مرتبه به شبکه عصبی آموزش داده می‌شود

number_of_users   = 711221     #   تعداد کل  کاربران موجود در دیتاست




total_loss = []

for episode in range(n_episodes):

   print(f"Episode {episode+1} in progress.....")

   #یاد گرفته شود    end_index    تا کاربر شماره ی     start_index      از کاربر شماره ی    
   start_index = np.random.randint( low=0   ,  high= number_of_users -500)
   end_index   = start_index + length_of_episode
   
   episode_loss =[]
   for userid in range(start_index , end_index ):
      
      loss_ = dqn.Learn_favorite_category(userid , n_iterations )

      episode_loss.append(loss_)

   episode_loss = np.mean(episode_loss)
   total_loss.append(episode_loss)

      
   if episode%10==0:
      dqn.soft_Update(dqn.target_model.trainable_variables,dqn.model.trainable_variables )
   
  


   print(f"episode {episode +1} finished. loss = {episode_loss} \n","="*50)





Episode 1 in progress.....
episode 1 finished. loss = 42.76100540161133 
Episode 2 in progress.....
episode 2 finished. loss = 122.8829116821289 
Episode 3 in progress.....
episode 3 finished. loss = 48.442771911621094 
Episode 4 in progress.....
episode 4 finished. loss = 47.33455276489258 
Episode 5 in progress.....
episode 5 finished. loss = 71.736328125 
Episode 6 in progress.....
episode 6 finished. loss = 146.98751831054688 
Episode 7 in progress.....
episode 7 finished. loss = 100.71116638183594 
Episode 8 in progress.....
episode 8 finished. loss = 158.2576141357422 
Episode 9 in progress.....
episode 9 finished. loss = 93.08625793457031 
Episode 10 in progress.....
episode 10 finished. loss = 222.7868194580078 
